# Allergens

--- Day 21: Allergen Assessment ---
You reach the train's last stop and the closest you can get to your vacation island without getting wet. There aren't even any boats here, but nothing can stop you now: you build a raft. You just need a few days' worth of food for your journey.

You don't speak the local language, so you can't read any ingredients lists. However, sometimes, allergens are listed in a language you do understand. You should be able to use this information to determine which ingredient contains which allergen and work out which foods are safe to take with you on your trip.

You start by compiling a list of foods (your puzzle input), one food per line. Each line includes that food's ingredients list followed by some or all of the allergens the food contains.

Each allergen is found in exactly one ingredient. Each ingredient contains zero or one allergen. Allergens aren't always marked; when they're listed (as in (contains nuts, shellfish) after an ingredients list), the ingredient that contains each listed allergen will be somewhere in the corresponding ingredients list. However, even if an allergen isn't listed, the ingredient that contains that allergen could still be present: maybe they forgot to label it, or maybe it was labeled in a language you don't know.

For example, consider the following list of foods:
```
mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)
```
The first food in the list has four ingredients (written in a language you don't understand): `mxmxvkd, kfcds, sqjhc, and nhms`. While the food might contain other allergens, a few allergens the food definitely contains are listed afterward: dairy and fish.

The first step is to determine which ingredients can't possibly contain any of the allergens in any food in your list. In the above example, none of the ingredients `kfcds, nhms, sbzzf, or trh` can contain an allergen. Counting the number of times any of these ingredients appear in any ingredients list produces 5: they all appear once each except `sbzzf`, which appears twice.

Determine which ingredients cannot possibly contain any of the allergens in your list. How many times do any of those ingredients appear?

Your puzzle answer was 2423.

## Second part

--- Part Two ---  
Now that you've isolated the inert ingredients, you should have enough information to figure out which ingredient contains which allergen.

In the above example:

`mxmxvkd` contains dairy.  
`sqjhc` contains fish.  
`fvjkl` contains soy.  
Arrange the ingredients alphabetically by their allergen and separate them by commas to produce your canonical dangerous ingredient list. (There should not be any spaces in your canonical dangerous ingredient list.) In the above example, this would be `mxmxvkd,sqjhc,fvjkl`.

Time to stock your raft with supplies. What is your canonical dangerous ingredient list?

In [1]:
import os
import time
import numpy as np
import itertools
import re
from collections import Counter

In [2]:
DAY = 'Day_21'
FILE_END = '_01.txt'
RUN_WITH = 'input' ## admitted values: sample input 
SAMPLE_DATA = '''mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)'''

In [3]:
start = time.time()

In [4]:

input_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))), "Inputs")
input_path_day = os.path.join(input_path, DAY)
file_path = os.path.join(input_path_day, DAY+ FILE_END)

### Read input

## Part I

In [5]:
def get_input():
    if RUN_WITH == 'input':
        with open(file_path,'r') as f:
            values = f.read()
    elif RUN_WITH == 'sample':
        values =  SAMPLE_DATA

    else:
        print("No valid input selected")
    values = values.split('\n')
    try:
        values.remove("")
    except:
        pass
    return values

def parse_instructions(values):
    instructions = {inst:{'ingredients':value.replace('(','').replace(')','').split('contains')[0].split(),
                      'allergens':list(map(str.strip, value.replace('(','').replace(')','').split('contains')[1].split(',')))}
               for inst, value in enumerate(values)
               }
    return instructions
    

def get_counters(instructions):
    all_ingredients = []
    all_allergens = []
    for k, v in instructions.items():
        all_ingredients.extend(v['ingredients'])
        all_allergens.extend(v['allergens'])
        
    all_ingredients = Counter(all_ingredients)
    all_allergens = Counter(all_allergens)
    
    allergens = {allergen:Counter() for allergen in all_allergens.keys()}
    for v in instructions.values():
        for allergen in v['allergens']:
            allergens[allergen].update(v['ingredients'])
            
    return all_ingredients, all_allergens, allergens

def get_safe(all_ingredients, all_allergens, allergens):
    # part I - remove ingredients that appear less times associated with the allergen than the allergen
    # e.g. dairy appears twice, so remove any ingredient from the "potentially dairy" that appears less than 2 times.
    remove = [] #tracks items to remove
    for allergen, value in all_allergens.items():
        for ingredient in allergens[allergen].keys():
            if allergens[allergen][ingredient] < value:
                remove.append((allergen, ingredient))
    #then removes them
    for item in remove:
        allergens[item[0]].pop(item[1])

    set_all = set(all_ingredients.keys())
    set_allergen = set([ingredient for allergen in allergens for ingredient in allergens[allergen]])
    set_safe = set_all.difference(set_allergen)
    
    return set_safe

def get_result(set_safe, all_ingre):
    result = 0
    for ingredient in set_safe:
        result += all_ingredients[ingredient]
    return result

def part_1():
    values = get_input()
    instructions = parse_instructions(values)
    all_ingredients, all_allergens, allergens = get_counters(instructions)
    set_safe = get_safe(all_ingredients, all_allergens, allergens)

    result = 0
    for ingredient in set_safe:
        result += all_ingredients[ingredient]
    return result

In [6]:
part_1()

2423

In [7]:
%%timeit
part_1()

3.19 ms ± 284 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Part II

In [8]:
def part_2():

    values = get_input()
    instructions = parse_instructions(values)
    all_ingredients, all_allergens, allergens = get_counters(instructions)
    set_safe = get_safe(all_ingredients, all_allergens, allergens)
    
    solved = {}
    while allergens:
        to_remove = []
        for allergen in allergens:
            if len(allergens[allergen]) == 1:
                solved[allergen] = list(allergens[allergen].keys())[0]
                to_remove.append((allergen, list(allergens[allergen].keys())[0]))
        for item in to_remove:
            allergens.pop(item[0])
            for allergen in allergens:
                if allergens[allergen].get(item[1]):
                    allergens[allergen].pop(item[1])
    
    answer = ''
    for key in sorted(solved.keys()):
        answer += solved[key]+','
    return answer[:-1]

In [9]:
part_2()

'jzzjz,bxkrd,pllzxb,gjddl,xfqnss,dzkb,vspv,dxvsp'

In [10]:
%%timeit
part_2()

3.25 ms ± 126 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
